In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

SEED = 1103

notebooks_dir = Path.cwd()
project_dir = notebooks_dir.parent
data_dir = project_dir / 'data' / 'raw'
text_data_path = data_dir / 'Subtask_2_train.json'
interim_dir = project_dir / 'data' / 'interim'
mel_dir = interim_dir / 'mel' 
models_dir = project_dir / 'models'

In [2]:
text_data = json.loads(text_data_path.read_text())
for i in range(len(text_data[0]['conversation'])):
    print(text_data[0]['conversation'][i])

{'utterance_ID': 1, 'text': 'Alright , so I am back in high school , I am standing in the middle of the cafeteria , and I realize I am totally naked .', 'speaker': 'Chandler', 'emotion': 'neutral', 'video_name': 'dia1utt1.mp4'}
{'utterance_ID': 2, 'text': 'Oh , yeah . Had that dream .', 'speaker': 'All', 'emotion': 'neutral', 'video_name': 'dia1utt2.mp4'}
{'utterance_ID': 3, 'text': 'Then I look down , and I realize there is a phone ... there .', 'speaker': 'Chandler', 'emotion': 'surprise', 'video_name': 'dia1utt3.mp4'}
{'utterance_ID': 4, 'text': 'Instead of ... ?', 'speaker': 'Joey', 'emotion': 'surprise', 'video_name': 'dia1utt4.mp4'}
{'utterance_ID': 5, 'text': 'That is right .', 'speaker': 'Chandler', 'emotion': 'anger', 'video_name': 'dia1utt5.mp4'}
{'utterance_ID': 6, 'text': 'Never had that dream .', 'speaker': 'Joey', 'emotion': 'neutral', 'video_name': 'dia1utt6.mp4'}
{'utterance_ID': 7, 'text': 'No .', 'speaker': 'Phoebe', 'emotion': 'neutral', 'video_name': 'dia1utt7.mp4'}

In [3]:
def get_data_from_json(json):
    text = []
    emotions = []
    filenames = []
    for line_idx in range(len(json)):
        text += [json[line_idx]['text']]
        emotions += [json[line_idx]['emotion']]
        filenames += [json[line_idx]['video_name'][:-4]]
    return text, emotions, filenames

convos = []
all_emotions = []
all_filenames = []
for convo_idx in range(len(text_data)):
    convo, emotions, filenames = get_data_from_json(text_data[convo_idx]['conversation'])
    convos += convo
    all_emotions += emotions
    all_filenames += filenames

data = {'filename': all_filenames, 'text' : convos, 'label': all_emotions}
text_data_dict = {'text' : convos, 'label': all_emotions}

In [4]:
df = pd.DataFrame.from_dict(text_data_dict)

labels = ['neutral', 'joy', 'surprise', 'anger', 'fear', 'disgust', 'sadness']
label2id = {label: index for index, label in enumerate(labels)}
id2label = {index: label for index, label in enumerate(labels)}

df['label'] = df['label'].apply(lambda label: label2id[label])

df.head()

,text,label
0,"Alright , so I am back in high school , I am s...",0
1,"Oh , yeah . Had that dream .",0
2,"Then I look down , and I realize there is a ph...",2
3,Instead of ... ?,2
4,That is right .,3


In [10]:
import transformers
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

RuntimeError: Failed to import transformers.models.distilbert.tokenization_distilbert because of the following error (look up to see its traceback):
cannot import name '__version__' from 'tokenizers.tokenizers' (/Users/teodorastereciu/Documents/bachelors-project/mc-ecpe/.venv/lib/python3.10/site-packages/tokenizers/tokenizers.cpython-310-darwin.so)

In [ ]:
import tensorflow as tf
X_tf = [tokenizer(text, padding='max_length', max_length = 512, truncation=True)['input_ids'] for text in X]
X_tf = np.array(X_tf, dtype='int32')
# Train/test split
from sklearn.model_selection import train_test_split
X_tf_train, X_tf_test, y_tf_train, y_tf_test = train_test_split(X_tf, y, test_size=0.3, random_state=42, stratify=y)
print('Shape of training data: ',X_tf_train.shape)
print('Shape of test data: ',X_tf_test.shape)
